In [1]:
from importlib import reload
import platform, os, sys, datetime, re, itertools
from os.path import join
from glob import glob
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from cvt.TrAQ.Tank import Tank
import cvt.utils as utils

tank_diameter_vs_age = { 7:9.6, 14:10.4, 21:12.8, 28:17.7, 42:33.8 }
plt.rcParams['figure.dpi'] = 150
# plt.rcParams['figure.figsize'] = 9,6

# Load trials

In [2]:
trial_files = sorted(glob('../tracking/full_20-07-14/*/trial.pik')) #[:1]
print(len(trial_files))

95


In [3]:
trials = {}
for trial_file in trial_files:
    
    # Parse the trial's name.
    trial_dir  = os.path.dirname(trial_file)
    trial_name = os.path.basename(trial_dir)
    pop,_,age,group,n_ind = trial_name.split('_')[:5]
    age   = int(age[:-3])
#     n_ind = int(re.findall('\d+',n_ind)[0]) # The trial dictionary already contains n_ind.
    trial = utils.load_pik(trial_file)
    locals().update(trial)
    
    # Load the tracking data, originally a numpy.array, into a pandas.DataFrame.
    fish_list = range(n_ind)
    df = pd.DataFrame(data.reshape((data.shape[0],-1)))
    df.columns = pd.MultiIndex.from_product([ fish_list, ['x_px','y_px','ang','area'] ])
    df.index = pd.Index(frame_list.astype(float)/fps, name='time')
    
    # Create figure directory inside the tracking directory.
    fig_dir = os.path.join(trial_dir,'figures')
    if not os.path.exists(fig_dir):
        os.mkdir(fig_dir)
    
    trials[trial_file] = { k:globals()[k] for k in ['trial_dir', 'trial_name', 'fig_dir', 
                                                    'pop', 'age', 'group','n_ind',
                                                    'df', 'fish_list'] }

# Analyze


### Compute spatial and kinematic quantities

In [4]:
def derivative(series):
#     # First order forward derivative.
#     x,y = series.index,series.values
#     return pd.Series((y[1:]-y[:-1])/(x[1:]-x[:-1]),x[1:])
    # Second order centered derivative.
#     x,y = series.index,series.values
#     return pd.Series((y[2:]-y[:-2])/(x[2:]-x[:-2]),x[1:-1])
    return pd.Series(np.gradient(series,series.index),index=series.index)


for trial_file,trial in trials.items():
    locals().update(trial)
    
    # Convert pixels to cm.
    tank.r_cm = tank_diameter_vs_age[age]/2
    a = tank.r_cm/tank.r_px
    for i in range(n_ind):
        df[i,'x'] =  a*(df[i,'x_px']-tank.x_px)
        df[i,'y'] = -a*(df[i,'y_px']-tank.y_px)
    
    # Compute spatial and kinematic quantities.
    for f in fish_list:
        df[f,'d_wall'] = tank.r_cm - np.hypot(df[f,'x'],df[f,'y'])
        df[f,'v_x']    = derivative(df[f,'x'])
        df[f,'v_y']    = derivative(df[f,'y'])
        df[f,'v']      = np.hypot(df[f,'v_x'],df[f,'v_y'])
        # Make sure the difference between the current angle and the last
        # non-NaN angle is never larger than 2*pi, then take the derivative.
        I              = ~df[f,'ang'].isna()
        df[f,'ang'][I] = np.unwrap(df[f,'ang'][I])
        df[f,'v_ang']  = derivative(df[f,'ang'])
    
    # Reorder columns.
    # df = df.sort_index(axis=1) # Sort alphabetically, which also sorts by fish.
    columns = [ (f,q) for f in fish_list for q in ['x_px', 'y_px', 'area', 'ang', 'v_ang', 
                                                   'x', 'y', 'v_x', 'v_y', 'v', 'd_wall'] ]
    df = df.reindex(columns=columns)
    

### Perform cuts

*TODO*: Add an entry to `valid_fraction` to report the amount of NaN prior to any cut.

In [5]:
for trial_file,trial in trials.items():
    locals().update(trial)
    
    cut_ranges = { 'd_wall': [0,tank.r_cm], 
                   'v':      [0,np.inf], 
                   'v_ang':  [-np.inf,np.inf] }
    
    cut_label  = ', '.join([ f'{v[0]}<{k.split(" ")[0]}<{v[1]}' for k,v in cut_ranges.items() ])
    columns    = pd.MultiIndex.from_product([ fish_list, list(cut_ranges.keys())+['final'] ])
    cuts       = pd.DataFrame(index=df.index, columns=columns, dtype=bool)
    
    # Compute cuts.
    valid_fraction = {}
    for f in fish_list:
        for c,(vmin,vmax) in cut_ranges.items():
            cuts[f,c]  = (df[f,c]>vmin) & (df[f,c]<vmax)
        cuts[f,'final'] = cuts[f].all(axis=1)
        
    # Compute the fraction of points that made it through each cut.
    valid_fraction = {}
    for c in list(cut_ranges.keys())+['final']:
        B = [cuts[f,c] for f in fish_list]
        valid_fraction[c] = sum([b.sum() for b in B])/sum([len(b) for b in B])
    
    # Create filtered dataframe, i.e., perform the cut.
    df2 = df.copy()
    for f in fish_list:
        df2.loc[~cuts[f,'final'],f] = np.nan
    
    trial.update({ k:globals()[k] for k in ['cut_ranges', 'cut_label', 'cuts', 'valid_fraction', 'df2'] })


# Plot

*TODO*? Use an "acclimation cut" (dismiss the first xx minutes).

### Trajectories

In [6]:
for trial_file,trial in trials.items():
#     print(trial_file)
    locals().update(trial)
    
    plt.figure(figsize=(6,)*2)
    circle = plt.Circle( (tank.x_px,tank.y_px), tank.r_px,
                         facecolor='None', edgecolor='k', lw=0.5 )
    plt.gca().add_patch(circle)
    for fish in range(data.shape[1]):
        x,y,theta,area = data[:,fish].T
        plt.scatter(x,y,s=0.1,linewidths=0,label=str(fish))
#         plt.plot(x,y,lw=0.5,label=str(fish))
    plt.axis('equal')
    plt.gca().yaxis.set_inverted(True)
    plt.legend()
    
    plt.suptitle(trial_name)
    plt.savefig(os.path.join(fig_dir,f'trajectories.png'))
 #     plt.show()
    plt.close()


In [7]:
# plt.close('all')

### Angle dynamics

In [8]:
for trial_file,trial in trials.items():
#     print(trial_file)
    locals().update(trial)
    
    plt.figure(figsize=(12,6))
    for f in fish_list:
        a  = df[f,'ang']
        plt.plot(a.index,a,label=f) #,marker='.')
    plt.xlabel('Time (s)')
    plt.ylabel('Angle (rad)')
    plt.legend()
    
    plt.suptitle(trial_name)
    plt.savefig(os.path.join(fig_dir,f'angle-vs-time.png'))
 #     plt.show()
    plt.close()


### Valid fraction

In [9]:
for trial_file,trial in trials.items():
#     print(trial_file)
    locals().update(trial)
    
    bp = plt.bar(*zip(*valid_fraction.items()))
    for bar in bp:
        h,x,w = bar.get_height(),bar.get_x(),bar.get_width()
        plt.annotate(f'{h:.2f}', xy=(x+w/2,1.01), ha='center', va='bottom')
    plt.ylim(0,1.1)
    plt.ylabel('Valid fraction')
    
    plt.title(cut_label)
    plt.suptitle(trial_name)
    plt.savefig(os.path.join(fig_dir,f'valid_fraction.png'))
#     plt.show()
    plt.close()


### Distance to the wall distribution

In [10]:
for trial_file,trial in trials.items():
#     print(trial_file)
    locals().update(trial)
    
    quantity = 'd_wall'
    unit = 'cm'
    bins = np.linspace(0,tank.r_cm,20)
    values = df2.loc[:,(slice(None),quantity)].values.flatten()
    plt.hist(values,bins=bins)
    plt.xlabel(f'{quantity} ({unit})')
    plt.ylabel('frequency')
    
    plt.title(cut_label)
    plt.suptitle(trial_name)
    plt.savefig(os.path.join(fig_dir,f'dwall__histogram.png'))
#     plt.show()
    plt.close()


### Velocity distribution

In [11]:
for trial_file,trial in trials.items():
#     print(trial_file)
    locals().update(trial)
    
    quantity = 'v'
    unit = 'cm/s'
    bins = 30
    values = df2.loc[:,(slice(None),quantity)].values.flatten()
    plt.hist(values,bins=bins)
    plt.xlabel(f'{quantity} ({unit})')
    plt.ylabel('frequency')
    plt.yscale('log')

    plt.title(cut_label)
    plt.suptitle(trial_name)
    plt.savefig(os.path.join(fig_dir,f'v__histogram.png'))
#     plt.show()
    plt.close()


/home/yaouen/software/miniconda3/envs/main/lib/python3.6/site-packages/numpy/lib/histograms.py:839: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= first_edge)
/home/yaouen/software/miniconda3/envs/main/lib/python3.6/site-packages/numpy/lib/histograms.py:840: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= last_edge)


### Angular velocity distribution

In [12]:
for trial_file,trial in trials.items():
#     print(trial_file)
    locals().update(trial)
    
    quantity = 'v_ang'
    unit = 'rad/s'
    bins = 50

    values = df2.loc[:,(slice(None),quantity)].values.flatten()
    plt.hist(values,bins=bins)
    plt.xlabel(f'{quantity} ({unit})')
    plt.ylabel('frequency')
    plt.yscale('log')

    plt.title(cut_label)
    plt.suptitle(trial_name)
    plt.savefig(os.path.join(fig_dir,f'ang_v__histogram.png'))
#     plt.show()
    plt.close()


In [13]:
# ''' Analyze instances of unusually high velocity. '''

# # At 30 fps, |v_ang|=30 (about where the rare peaks start) 
# # corresponds to about pi/3 in one frame.
# print('v_ang for pi/3 in (1/30) second:',np.pi/3*fps)

# print('Instances of unusually high v_ang:')
# for f in fish_list:
#     ang_diff  = df[f,'ang'].diff()
#     I = np.nonzero(np.absolute(ang_diff.values)>1)[0]
#     for i in I[:5]:
#         display(df[f,'ang'].iloc[i-1:i+2])

### Joint pair distance-pair angle distribution

In [17]:
bins_d = np.linspace(0,2*tank.r_cm,15) # pair distance bins
bins_a = np.linspace(0,np.pi,15)       # pair angle bins

for trial_file,trial in trials.items():
#     print(trial_file)
    locals().update(trial)
    
    if n_ind<2:
        continue
    
    H = [] # np.zeros((len(bins_d)-1,len(bins_a)-1))
    for f1,f2 in itertools.combinations(fish_list,2):
        dx,dy    = df[f1,'x']-df[f2,'x'],df[f1,'y']-df[f2,'y']
        d        = np.hypot(dx,dy)
        a        = df[f1,'ang']-df[f2,'ang']
        a        = np.absolute(a - 2*np.pi*np.rint(a/(2*np.pi)))
        h,_,_    = np.histogram2d(d,a,bins=(bins_d,bins_a),normed=True)
        H.append(h)

    H = np.mean(H,axis=0)
    plt.pcolormesh(bins_d,bins_a*180/np.pi,H.T)
    plt.xlabel('pair distance (cm)')
    plt.ylabel('pair angle (deg)')
    plt.colorbar()
    
    plt.title(cut_label)
    plt.suptitle(trial_name)
    plt.savefig(os.path.join(fig_dir,f'distance-angle__2d-histogram.png'))
#     plt.show()
    plt.close()
    

# Clean up

In [18]:
# for trial_file,trial in trials.items():
#     locals().update(trial)
    
#     # Delete figure directory and its contents.
# #     os.remove(fig_dir)
    